In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping

from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [14]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
#from tensorflow.keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image  

In [53]:
import pandas as pd
import numpy as np

In [3]:
csv_file = pd.read_csv('driver_imgs_list.csv')

In [6]:
csv_file.head()

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg


In [9]:
images_dir = 'imgs/train'

In [25]:
batch_size = 16
train_input_shape = (64, 64, 3)
train_datagen = ImageDataGenerator(validation_split=0.2,
                                   rescale=1./255.,
                                   #rotation_range=45,
                                   #width_shift_range=0.5,
                                   #height_shift_range=0.5,
                                   shear_range=5,
                                   #zoom_range=0.7,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                  )

In [26]:
train_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    shuffle=True
                                                   )


valid_generator = train_datagen.flow_from_directory(directory=images_dir,
                                                    class_mode='categorical',
                                                    target_size=train_input_shape[0:2],
                                                    batch_size=batch_size,
                                                    subset="validation",
                                                    shuffle=True)

Found 17943 images belonging to 10 classes.
Found 4481 images belonging to 10 classes.


In [45]:
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=train_input_shape, kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu', kernel_initializer='glorot_normal'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu', kernel_initializer='glorot_normal'))
model.add(Dropout(0.2))
model.add(Dense(10, activation='softmax', kernel_initializer='glorot_normal'))


model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_20 (Conv2D)           (None, 64, 64, 64)        832       
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 32, 32, 128)       32896     
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 16, 16, 256)       131328    
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 8, 8, 512)        

In [46]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [47]:
n_epoch = 20

early_stop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, 
                           mode='auto', restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, 
                              verbose=1, mode='auto')

In [48]:

history2 = model.fit(train_generator,
                              validation_data=valid_generator,
                              epochs=n_epoch,
                              shuffle=True,
                              verbose=1,
                                callbacks=[reduce_lr, early_stop])

Epoch 1/20
1122/1122 [==============================] - 143s 127ms/step - loss: 1.8848 - accuracy: 0.2899 - val_loss: 1.2292 - val_accuracy: 0.5570
Epoch 2/20
1122/1122 [==============================] - 141s 126ms/step - loss: 0.8970 - accuracy: 0.6901 - val_loss: 0.6262 - val_accuracy: 0.7791
Epoch 3/20
1122/1122 [==============================] - 141s 126ms/step - loss: 0.5348 - accuracy: 0.8274 - val_loss: 0.3214 - val_accuracy: 0.9007
Epoch 4/20
1122/1122 [==============================] - 141s 126ms/step - loss: 0.3966 - accuracy: 0.8728 - val_loss: 0.2995 - val_accuracy: 0.9016
Epoch 5/20
1122/1122 [==============================] - 141s 125ms/step - loss: 0.3459 - accuracy: 0.8969 - val_loss: 0.2133 - val_accuracy: 0.9331
Epoch 6/20
1122/1122 [==============================] - 141s 126ms/step - loss: 0.3128 - accuracy: 0.9062 - val_loss: 0.1979 - val_accuracy: 0.9411
Epoch 7/20
1122/1122 [==============================] - 141s 125ms/step - loss: 0.2980 - accuracy: 0.9127 - val_

In [49]:
model.save('final_model')

INFO:tensorflow:Assets written to: final_model/assets


In [50]:
from tensorflow.keras.models import load_model

In [51]:
saved_model = load_model('final_model')

In [69]:
from PIL import Image
test_imag = Image.open('imgs/test/img_1.jpg') 
test_image = test_imag.resize([64,64])

test_image = np.array(test_image).reshape(64,64,3)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis=0)

prediction = saved_model.predict(test_image)

In [70]:
test_imag.show()

In [71]:
prediction_probability = np.amax(prediction)
prediction_idx = np.argmax(prediction)

In [72]:
prediction_idx

5

In [73]:
train_generator.class_indices

{'c0': 0,
 'c1': 1,
 'c2': 2,
 'c3': 3,
 'c4': 4,
 'c5': 5,
 'c6': 6,
 'c7': 7,
 'c8': 8,
 'c9': 9}